In [ ]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob

import glob
from bs4 import BeautifulSoup
from tqdm import tqdm
import cv2
%matplotlib inline





In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


# Get image as numpy array
def load_image(name, path):
    img_path =os.path.join( path , name + '.png')
#     print(img_path)
    img = cv2.imread(img_path)
    return img

# Plot numpy array
def plot_image(img):
    plt.figure(figsize = (10,10)) #figsize = (20,20)
    plt.imshow(img)
    plt.title(img.shape)
    
# Plot a grid of examples
def plot_grid(img_names, img_root, rows=5, cols=5):
    fig = plt.figure(figsize=(25,25))
    
    for i,name in enumerate(img_names):
        fig.add_subplot(rows,cols,i+1)
        img = load_image(name, img_root)
        plot_image(img)
        
    plt.show()

def class_to_color(class_id):
    colors = [(255,0,0),(0,255,0),(0,0,255),
              (100,255,100)]
    return colors[class_id]


def draw_bounding_box(img, annotation):
    if annotation.isnull().values.any():
        return
    
    x_min, y_min = int(annotation['xmin']), int(annotation['ymin'])
    x_max, y_max = int(annotation['xmax']), int(annotation['ymax'])
    
    class_id = int(annotation['class_id'])
    color = class_to_color(class_id)
    
    cv2.rectangle(img,(x_min,y_min),(x_max,y_max), color, 2)   #.permute(1,2,0).cpu().numpy()
    
# draw all annotation bounding boxes on an image
def annotate_image(img, name, all_annotations):
    annotations = all_annotations[all_annotations['image_id'] == name]
    for index, row in annotations.iterrows():
        draw_bounding_box(img, row)

In [ ]:
SEED = 42

In [ ]:
# !ls ../input/cotton-data/cloud-object-storage-tw-cos-standard-tqh
# pd.Series(dff['file_name'].unique())
# dff['file_name'].unique()[:30]

seed_everything(SEED)


dir_path = "../input/cotton-data/cloud-object-storage-tw-cos-standard-tqh" 
df = {"file_name":[],"width":[],"height":[],"depth":[],"label":[],"pose":[],"truncated":[],"difficult":[],"xmin":[],"ymin":[],"xmax":[],"ymax":[]}
for path in tqdm(glob.glob(dir_path +"/*.xml")):
    with open(path, 'r') as f:
        data = f.read()
    Bs_data = BeautifulSoup(data, "xml")
    objects = Bs_data.find_all('object')
    for obj in objects:
        df["width"].append(Bs_data.find('size').find('width').contents[0])
        df["height"].append(Bs_data.find('size').find('height').contents[0])
        df["depth"].append(Bs_data.find('size').find('depth').contents[0])
        
        df["file_name"].append(os.path.basename(path).split(".")[0])
        df["label"].append(obj.find('name').contents[0])
        df["pose"].append(obj.find('pose').contents[0])
        df["truncated"].append(obj.find('truncated').contents[0])
        df["difficult"].append(obj.find('difficult').contents[0])
        df["xmin"].append(obj.find('bndbox').find('xmin').contents[0])
        df["ymin"].append(obj.find('bndbox').find('ymin').contents[0])
        df["xmax"].append(obj.find('bndbox').find('xmax').contents[0])
        df["ymax"].append(obj.find('bndbox').find('ymax').contents[0])
 
dff = pd.DataFrame.from_dict(df)
dff = dff[dff['label']!="Hand"]
dff = dff[dff['label']!="Untitled Label"]

dff = dff[['file_name', 'width', 'height', 'label',
        'xmin', 'ymin', 'xmax', 'ymax']]

dff['class_id'] = 0
dff['class_id'][dff['label']=="Tray"]=1
dff['class_id'][dff['label']=="Plastic"]=2


dff.rename(columns={'file_name':'image_id','label':'source'}, inplace=True)

dff["x"]=(1024*(dff["xmin"].astype(int)/320)).astype(int)
dff["y"]=(1024*(dff["ymin"].astype(int)/240)).astype(int)
dff["w"]=(1024*(dff["xmax"].astype(int)/320))-dff["x"]
dff["h"]=(1024*(dff["ymax"].astype(int)/240))-dff["y"]

dff["x"]=dff["x"].astype(int)
dff["y"]=dff["y"].astype(int)
dff["w"]=dff["w"].astype(int)
dff["h"]=dff["h"].astype(int)

dff['width']=1024
dff['height']=1024

marking = dff[['image_id', 'width', 'height', 'source', 'x', 'y', 'w', 'h']]

In [ ]:
dff

In [ ]:


def plot_grid(dff, img_root, rows=2, cols=2):
    img_names = pd.Series(dff['image_id'].unique())
    fig, axs = plt.subplots(rows, cols, figsize=(32,32))
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(img_names), size=1)[0]
            name = dff.iloc[idx].image_id
            img = load_image(name, img_root)
            annotate_image(img, name, dff)
            axs[row, col].imshow(img)

In [ ]:
IMG_ROOT = "../input/cotton-data/cloud-object-storage-tw-cos-standard-tqh"
plot_grid(dff, IMG_ROOT)

In [ ]:
#######   convert to jpg

dir_path = "../input/cotton-data/cloud-object-storage-tw-cos-standard-tqh"
# image_name = "img_1264.png"
destination_dir_path= "/kaggle/working/train"

!mkdir train

width = 1024
height = 1024
dim = (width, height)

for path in tqdm(glob.glob(dir_path +"/*.png")):
    image_name = os.path.basename(path).split(".")[0]
    img = cv2.imread(os.path.join(dir_path,image_name+".png"), cv2.IMREAD_UNCHANGED)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(os.path.join(destination_dir_path,image_name+".jpg"), resized, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = marking[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

In [ ]:
marking

In [ ]:
marking.source.value_counts()

In [ ]:
df_folds.source.value_counts()

In [ ]:
df_folds

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.3),   #0.5
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.3),  #0.9
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.3),   #p=0.9
            ],p=0.5),  #p=0.9
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),  #p=0.5
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, fill_value=0, p=0.3),    # max_h_size=64, max_w_size=64, fill_value=0, p=0.5
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:

marking.loc[:, 'label'] = 0
marking.loc[marking["source"]=="Cotton", 'label'] = 1
marking.loc[marking["source"]=="Tray", 'label'] = 2
marking.loc[marking["source"]=="Plastic", 'label'] = 3

In [ ]:


TRAIN_ROOT_PATH ="/kaggle/working/train"

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
#         if self.test or random.random() > 0.5:
#             image, boxes = self.load_image_and_boxes(index)
#         else:
#             image, boxes = self.load_cutmix_image_and_boxes(index)
        
        image, boxes,labels = self.load_image_and_boxes(index)
        # there is only one class
#         labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = torch.from_numpy(labels)
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        labels = records['label'].values
        
        return image, boxes,labels

    def load_cutmix_image_and_boxes(self, index, imsize=1024):
        """ 
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia 
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """
        w, h = imsize, imsize
        s = imsize // 2
    
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, self.image_ids.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32)
        result_boxes = []

        for i, index in enumerate(indexes):
            image, boxes = self.load_image_and_boxes(index)
            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)]
        return result_image, result_boxes

In [ ]:
fold_number = 0

train_dataset = DatasetRetriever(
    image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
    marking=marking,
    transforms=get_train_transforms(),
    test=False,
)

validation_dataset = DatasetRetriever(
    image_ids=df_folds[df_folds['fold'] == fold_number].index.values,
    marking=marking,
    transforms=get_valid_transforms(),
    test=True,
)

In [ ]:
image, target, image_id = train_dataset[1]
boxes = target['boxes'].cpu().numpy().astype(np.int32)

numpy_image = image.permute(1,2,0).cpu().numpy()

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(numpy_image, (box[1], box[0]), (box[3],  box[2]), (0, 1, 0), 2)
    
ax.set_axis_off()
ax.imshow(numpy_image);

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import warnings

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
#                 for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
#                     os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]

                loss, _, _ = self.model(images, boxes, labels)
                summary_loss.update(loss.detach().item(), batch_size)

        return summary_loss

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]

            self.optimizer.zero_grad()
            
            loss, _, _ = self.model(images, boxes, labels)
            
            loss.backward()

            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [ ]:
class TrainGlobalConfig:
    num_workers = 2
    batch_size = 8      #4   
    n_epochs = 80 # n_epochs = 40
    lr = 0.005

    folder = 'effdet5-cutmix-augmix'

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TrainGlobalConfig.batch_size,
    sampler=RandomSampler(train_dataset),
    pin_memory=False,
    drop_last=True,
    num_workers=TrainGlobalConfig.num_workers,
    collate_fn=collate_fn,
)
val_loader = torch.utils.data.DataLoader(
    validation_dataset, 
    batch_size=TrainGlobalConfig.batch_size,
    num_workers=TrainGlobalConfig.num_workers,
    shuffle=False,
    sampler=SequentialSampler(validation_dataset),
    pin_memory=False,
    collate_fn=collate_fn,
)

In [ ]:


def run_training():
    device = torch.device('cuda:0')
    net.to(device)



    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader, val_loader)
    return fitter.model

In [ ]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet

def get_net():
    config = get_efficientdet_config('tf_efficientdet_d0')  #tf_efficientdet_d5  
    net = EfficientDet(config, pretrained_backbone=True)
#     checkpoint = torch.load('../input/efficientdet/efficientdet_d5-ef44aea8.pth')
#     net.load_state_dict(checkpoint)
    config.num_classes = 3
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    return DetBenchTrain(net, config)

net = get_net()

In [ ]:
!ls effdet5-cutmix-augmix

In [ ]:
model = run_training()

In [ ]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
# validation_dataset.image_ids

In [ ]:
# # validation_dataset.image_ids

# temp_list_of_ids = ['img_7094', 'img_7284', 'img_7186', 'img_1468', 'img_7287',
#        'img_1747', 'img_1477', 'img_7113', 'img_1811', 'img_1873',
#        'img_1861', 'img_1821', 'img_1626', 'img_1628', 'img_7145',
#        'img_7107', 'img_1820', 'img_1487', 'img_1458', 'img_1451']

In [ ]:
# DATA_ROOT_PATH = '../input/global-wheatl-detection/test'
DATA_ROOT_PATH = "/kaggle/working/train"



class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
dataset = DatasetRetriever(
    image_ids=validation_dataset.image_ids,
    transforms=get_valid_transforms()
)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
# !pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
# !pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
# !ls ../input/weightedboxesfusion

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")

from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet

In [ ]:
!ls

In [ ]:
!ls effdet5-cutmix-augmix

In [ ]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d0')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 3
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

# '../input/wheat-effdet5-fold0-best-checkpoint/fold0-best-all-states.bin'

path = 'effdet5-cutmix-augmix/best-checkpoint-028epoch.bin'
# path = 'effdet5-cutmix-augmix/last-checkpoint.bin'
net = load_net(path)

In [ ]:
def make_predictions(images, score_threshold=0.22):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4] 
            labels = det[i].detach().cpu().numpy()[:,5]
            scores = det[i].detach().cpu().numpy()[:,4]
            indexes = np.where(scores > score_threshold)[0]
#             labels = labels[indexes]
#             boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                 'labels':labels[indexes]
            })
    return [predictions]

def run_wbf(predictions, image_index, image_size=512, iou_thr=0.35, skip_box_thr=0.3, weights=None):   #iou_thr=0.44, skip_box_thr=0.43
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
#     labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    labels = [prediction[image_index]['labels'].tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:
# labels

In [ ]:
# boxes

In [ ]:

import matplotlib.pyplot as plt

for j, (images, image_ids) in enumerate(data_loader):
    if j==6: break

    predictions = make_predictions(images)

    i = 1
    sample = images[i].permute(1,2,0).cpu().numpy()
    
    boxes, scores, labels = run_wbf(predictions, image_index=i)
    boxes = boxes.astype(np.int32).clip(min=0, max=511)
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 23))
    
    
    for i,box in enumerate(boxes):
        class_id = int(labels[i])
        color = class_to_color(class_id)
        cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), color, 1)
        
    ax.set_axis_off()
    ax.imshow(sample);

In [ ]:

# dfg = {"image_id":[],"xmin":[],"ymin":[],"xmax":[],"ymax":[],"label":[],"score":[]}

# for j, (images, image_ids) in enumerate(data_loader):
    
#     predictions = make_predictions(images)
#     for i in range(2):
          
#         try:   
#             boxes, scores, labels = run_wbf(predictions, image_index=i)
#             boxes = boxes.astype(np.int32).clip(min=0, max=511)
            
#             for k in range(len(labels)):
#                 dfg['image_id'].append(image_ids[i])
#                 dfg['xmin'].append(boxes[k][0])
#                 dfg['ymin'].append(boxes[k][1])
#                 dfg['xmax'].append(boxes[k][2])
#                 dfg['ymax'].append(boxes[k][3])
#                 dfg['label'].append(labels[k])
#                 dfg['score'].append(scores[k])
#         except: pass
# df = pd.DataFrame.from_dict(dfg)
# df['class_id']=df['label'].astype(int)

In [ ]:
# dff

In [ ]:
# df

In [ ]:


# def plot_grid(dff, img_root, rows=3, cols=3):
#     img_names = pd.Series(dff['image_id'].unique())
#     fig, axs = plt.subplots(rows, cols, figsize=(20,20))
#     for row in range(rows):
#         for col in range(cols):
#             idx = np.random.randint(len(img_names), size=1)[0]
#             name = dff.iloc[idx].image_id
#             img = load_image(name, img_root)
#             annotate_image(img, name, dff)
#             axs[row, col].imshow(img)

In [ ]:



# predictions = make_predictions(images)

# i = 1
# sample = images[i].permute(1,2,0).cpu().numpy()

# boxes, scores, labels = run_wbf(predictions, image_index=i)
# boxes = boxes.astype(np.int32).clip(min=0, max=511)

# fig, ax = plt.subplots(1, 1, figsize=(16, 8))

# for box in boxes:
#     cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
# ax.set_axis_off()
# ax.imshow(sample);

In [ ]:
# np.random.randint(len(pd.Series(df['image_id'].unique())), size=1)[0]

In [ ]:
# df.iloc[0].image_id

In [ ]:
# pd.Series(df['image_id'].unique())

In [ ]:
# IMG_ROOT = "../input/cotton-data/cloud-object-storage-tw-cos-standard-tqh"

# plot_grid(df, IMG_ROOT)

In [ ]:
# plot_grid(dff, IMG_ROOT)